<a href="https://colab.research.google.com/github/alex-jk/painting-lora-finetune/blob/main/paintings_lora_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip -O paintings.zip
!unzip -o paintings.zip -d paintings/

--2025-04-07 18:08:04--  https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7164316 (6.8M) [application/zip]
Saving to: ‘paintings.zip’

paintings.zip       100%[===================>]   6.83M  --.-KB/s    in 0.08s   

2025-04-07 18:08:04 (90.4 MB/s) - ‘paintings.zip’ saved [7164316/7164316]

Archive:  paintings.zip
  inflating: paintings/paintings/17030-islands-on-kabenung-40x60-brigitte-granton_orig.jpeg  
  inflating: paintings/paintings/23060-Setting-in-a-Grand-Way-Grand-Lake-30X60_-3240.jpeg  
  inflating: paintings/paintings/24055-Sunswept-Opeongo-40x48_-oil-Algonquin-G-539x443.jpeg  
  inflating: paintings/paintings/30yl2012ivanhoethechute600.jpg  
  inflating: painti

In [2]:
!pip install xformers

In [3]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers.git

Found existing installation: diffusers 0.33.0.dev0
Uninstalling diffusers-0.33.0.dev0:
  Successfully uninstalled diffusers-0.33.0.dev0
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-w2jd499q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-w2jd499q
  Resolved https://github.com/huggingface/diffusers.git to commit 506f39af3a7b533209cc96f1732fff347070bdbd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3544299 sha256=812f819c19d601719ce2d855911b580dd265f49e142109b187c0c10e66431610
  Stored in directory: /tmp/pip-ephem-wheel-cache-j4dt26fv/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers


In [8]:
import diffusers
print(diffusers.__version__)

from diffusers import StableDiffusionPipeline, UNet2DConditionModel
import torch

0.33.0.dev0


In [5]:
# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [15]:
# Enable LoRA training in the U-Net
pipe.unet.enable_gradient_checkpointing()
pipe.enable_xformers_memory_efficient_attention()

In [17]:
# Get a reference to the UNet model
unet = pipe.unet
print("UNet reference obtained.")
# --- End of your existing code + UNet Reference ---

# --- NEXT STEP (Corrected LoraConfig) ---

# 1. Import necessary classes from PEFT
from peft import LoraConfig # TaskType is no longer needed here

# 2. Define the LoRA configuration object
#    Omitting task_type as it's not required for unet.add_adapter()
#    and the previous enum value was incorrect.
lora_config = LoraConfig(
    r=4,
    lora_alpha=4,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.05,
    bias="none",
    # task_type=... # REMOVED - Not needed here and caused AttributeError
)

print("LoRA Config defined (without task_type).")
# --- End of NEXT STEP ---

UNet reference obtained.
LoRA Config defined (without task_type).


In [18]:
# 3. Add the LoRA adapter to the UNet model using the defined config
#    The 'adapter_name' is optional but good practice if you plan
#    to manage multiple adapters later.
adapter_name = "my_style_lora" # You can choose any descriptive name
unet.add_adapter(lora_config, adapter_name=adapter_name)

print(f"LoRA adapter '{adapter_name}' added to the UNet.")

# Optional but recommended: Verify trainable parameters
# This helps confirm that only the LoRA weights are set to be trained.
trainable_params = 0
all_param = 0
for name, param in unet.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        # Print trainable parameter names (optional, for debugging)
        # print(f"Trainable: {name}")
        trainable_params += param.numel()

print(
    f"\nTotal UNet params: {all_param:,}\n"
    f"Trainable params (LoRA): {trainable_params:,}\n"
    f"Trainable %: {100 * trainable_params / all_param:.4f}%"
)

LoRA adapter 'my_style_lora' added to the UNet.

Total UNet params: 860,318,148
Trainable params (LoRA): 797,184
Trainable %: 0.0927%


In [20]:
import torch.optim as optim

# 4. Set up the Optimizer
#    Filter the UNet's parameters to get only those that require gradients (the LoRA parameters).
lora_params_to_optimize = [p for p in unet.parameters() if p.requires_grad]
print(f"Number of parameter groups to optimize: {len(lora_params_to_optimize)}")

# Define common hyperparameters for the optimizer
learning_rate = 1e-4 # A common starting LR for LoRA training
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 1e-2
adam_epsilon = 1e-08

# Create the AdamW optimizer, passing only the LoRA parameters
optimizer = optim.AdamW(
    lora_params_to_optimize,
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=adam_weight_decay,
    eps=adam_epsilon,
)

print(f"Optimizer AdamW created with learning rate {learning_rate}.")

Number of parameter groups to optimize: 256
Optimizer AdamW created with learning rate 0.0001.
